In [ ]:
# Install FARM
!pip install torch==1.6.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html
!pip install farm==0.5.0
#!pip install transformers==2.8.0
import torch
import pandas as pd
from farm.modeling.adaptive_model import AdaptiveModel
from farm.data_handler.processor import Processor
from farm.data_handler.processor import TextClassificationProcessor
from transformers import AutoTokenizer
from farm.modeling.tokenization import Tokenizer
import csv

Looking in links: https://download.pytorch.org/whl/torch_stable.html


09/27/2021 16:16:19 - INFO - farm.modeling.prediction_head -   Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .


In [ ]:
# We need to fetch the right device to drive the growth of our model
# Make sure that you have gpu turned on in this notebook by going to
# Runtime>Change runtime type and select GPU as Hardware accelerator.

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Devices available: {}".format(device))

Devices available: cuda


# Here you can insert your created csv-file of the scraped comments of any YouTube video

In [ ]:
# test data without labels to see how our classifiers perform on unlabeled and unseen data
csv_file_name = 'InfoLing2_trump_video.csv'

test = pd.read_csv(f"../content/{csv_file_name}")
test

,Name,Comment,Time,Likes,Reply Count,Reply Author,Reply,Published,Updated
0,Diddy20022,"Today show and NBC, Trump won the election. It...",2021-09-27T11:21:41Z,0.0,0.0,NaN,NaN,NaN,NaN
1,G_ Arende,What a false narrative this channel is pushing 😳,2021-09-27T11:20:43Z,0.0,0.0,NaN,NaN,NaN,NaN
2,kalana nadun,If Biden won more than the count then voter co...,2021-09-27T11:16:21Z,0.0,0.0,NaN,NaN,NaN,NaN
3,Mo Stanley,Lol falsely said he won. If ppl are still bli...,2021-09-27T11:13:14Z,1.0,0.0,NaN,NaN,NaN,NaN
4,Abbadon380,Proven facts are labelled as Conspiracy theories,2021-09-27T11:12:20Z,0.0,0.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
2253,NaN,NaN,NaN,NaN,NaN,TJ Nelson,"@Jeremy Bottoms <a href=""https://youtu.be/QPXR...",2021-09-26T21:34:29Z,2021-09-26T21:34:29Z
2254,NaN,NaN,NaN,NaN,NaN,Jeremy Bottoms,@TJ Nelson lol couldn&#39;t win a debate so he...,2021-09-26T21:46:04Z,2021-09-26T21:46:04Z
2255,NaN,NaN,NaN,NaN,NaN,TJ Nelson,@Jeremy Bottoms Yeah bc H3H3 doesn&#39;t do po...,2021-09-26T22:19:23Z,2021-09-26T22:19:23Z
2256,NaN,NaN,NaN,NaN,NaN,TJ Nelson,"@Jeremy Bottoms you mean, Steven Coward",2021-09-26T22:20:04Z,2021-09-26T22:20:04Z


Because we have two separate columns "Comment" and "Reply" I want to merge them into one column



In [ ]:
comments = test['Comment']
comments_df = pd.DataFrame(columns=['text'])
comments_df['text'] = comments
# delete rows with NaN
comments_df.dropna(subset = ["text"], inplace=True)
comments_df.reset_index(inplace=True, drop=True)
#comments_df.drop('index', axis=1, inplace=True)
comments_df

09/27/2021 16:17:02 - INFO - numexpr.utils -   NumExpr defaulting to 4 threads.


,text
0,"Today show and NBC, Trump won the election. It..."
1,What a false narrative this channel is pushing 😳
2,If Biden won more than the count then voter co...
3,Lol falsely said he won. If ppl are still bli...
4,Proven facts are labelled as Conspiracy theories
...,...
863,Trump did win. Your type of propaganda will no...
864,"Trump is trying to destroy Democracy, why isn&..."
865,"<a href=""http://vae.fyi/"">vae.fyi\r</a><br>super"
866,He did win You’re pushing lies


In [ ]:
replies = test['Reply']
replies_df = pd.DataFrame(columns=['text'])
replies_df['text'] = replies
# delete rows with NaN
replies_df.dropna(subset = ["text"], inplace=True)
replies_df.reset_index(inplace=True, drop=True)
#replies_df.drop('index', axis=1, inplace=True)
replies_df

,text
0,Trump lost because he&#39;s nothing but a do n...
1,The majority of the entire world knew Trump lo...
2,Dasvidanya comrade. tebe nravitsya byt&#39; pr...
3,Where&#39;s your proof?
4,Yeah...no.
...,...
1385,"@Jeremy Bottoms <a href=""https://youtu.be/QPXR..."
1386,@TJ Nelson lol couldn&#39;t win a debate so he...
1387,@Jeremy Bottoms Yeah bc H3H3 doesn&#39;t do po...
1388,"@Jeremy Bottoms you mean, Steven Coward"


In [ ]:
frames = [comments_df, replies_df]

new_test = pd.concat(frames)

new_test.reset_index(inplace=True, drop=True)
new_test

,text
0,"Today show and NBC, Trump won the election. It..."
1,What a false narrative this channel is pushing 😳
2,If Biden won more than the count then voter co...
3,Lol falsely said he won. If ppl are still bli...
4,Proven facts are labelled as Conspiracy theories
...,...
2253,"@Jeremy Bottoms <a href=""https://youtu.be/QPXR..."
2254,@TJ Nelson lol couldn&#39;t win a debate so he...
2255,@Jeremy Bottoms Yeah bc H3H3 doesn&#39;t do po...
2256,"@Jeremy Bottoms you mean, Steven Coward"


Create dictionary for inference

In [ ]:
#test.set_index('id').T.to_dict('list')
#new_test = new_test.T.to_dict()
#new_test[1]
#basic_texts[1]
new_test.to_dict(orient='records')
new_test = new_test.to_dict(orient='records')

In [ ]:
new_test

NameError: ignored

In [ ]:
len(new_test)

2258

In [ ]:
# Test your model on a sample (Inference)
from farm.infer import Inferencer
from pprint import PrettyPrinter

#tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

#LABEL_LIST = ["0", "1"]

#processor = TextClassificationProcessor(tokenizer=tokenizer,
                                      #  max_seq_len=128,
                                      #  warmup = 600,
                                      #  train_filename="train.tsv",
                                      #  dev_filename="test_with_solutions.tsv",
                                      #  test_filename="impermium_verification_labels.tsv",
                                      #  data_dir="../content",
                                      #  label_list=LABEL_LIST,
                                      #  metric="f1_macro",
                                      #  text_column_name="Comment",
                                      #  label_column_name="Insult")

texts = [
    {"Text": "Martin is an idiot"},
    {"Text": "Martin Müller spielt Handball in Berlin"},
]


model = Inferencer.load('../content/roberta-base/')
#model = AdaptiveModel.load('../content/roberta-base', device=device)
#processor = Processor.load('../content/roberta-base/processor', device=device)
#model.connect_heads_with_processor(processor.tasks,require_labels=True)

#infer_model = Inferencer(processor=processor, model=model, task_type="text_classification", gpu=True)
#infer_model = Inferencer.load('../content/roberta-base', from_tf=True)

result = model.inference_from_dicts(dicts=new_test)
PrettyPrinter().pprint(result)

09/27/2021 16:17:50 - INFO - farm.utils -   device: cpu n_gpu: 0, distributed training: False, automatic mixed precision training: None
09/27/2021 16:17:53 - INFO - farm.modeling.adaptive_model -   Found files for loading 1 prediction heads
09/27/2021 16:17:53 - WARNING - farm.modeling.prediction_head -   `layer_dims` will be deprecated in future releases
09/27/2021 16:17:53 - INFO - farm.modeling.prediction_head -   Prediction head initialized with size [768, 2]
09/27/2021 16:17:53 - INFO - farm.modeling.prediction_head -   Using class weights for task 'text_classification': [0.6810969114303589, 1.8804762363433838]
09/27/2021 16:17:53 - INFO - farm.modeling.prediction_head -   Loading prediction head from ../content/roberta-base/prediction_head_0.bin
09/27/2021 16:17:53 - INFO - farm.modeling.tokenization -   Loading tokenizer of type 'RobertaTokenizer'
09/27/2021 16:17:53 - INFO - farm.data_handler.processor -   Initialized processor without tasks. Supply `metric` and `label_list` to

Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
                              'on the ballot. You just assume that they were '
                              'for Biden. What is funny is that they could '
                              'easily be for your golden idol which would mean '
                              'he lost by even more than the official count.',
                   'end': None,
                   'label': '0',
                   'probability': 0.99451184,
                   'start': None},
                  {'context': '@bradley mcmullen You are believing a lie!!  '
                              'Actually, the forensic audit conducted in '
                              'Maricopa County,  AZ uncovered 55,000 illegal '
                              'votes that should not have been counted and '
                              'will be removed.....most were cast for Biden!🤔',
                   'end': None,
                   'label': '0',
                 

In [ ]:
x = len(new_test)/4
print(x)
print(len(result))
result[567]
# 146 mal label 1 gezählt (physisch aus result nachgezählt) aber nur 107 Kommentare in Ausgabe enthalten
# der erste und letzte Kommentar mit label = 1 stimmen aber
result

564.5
568


[{'predictions': [{'context': 'Today show and NBC, Trump won the election. Its outlets like yours that lie and force people believe that he didn&#39;t. Your station is trash.',
    'end': None,
    'label': '0',
    'probability': 0.9002847,
    'start': None},
   {'context': 'What a false narrative this channel is pushing 😳',
    'end': None,
    'label': '0',
    'probability': 0.9769007,
    'start': None},
   {'context': 'If Biden won more than the count then voter countings are flawed',
    'end': None,
    'label': '0',
    'probability': 0.99586344,
    'start': None},
   {'context': 'Lol falsely said he won.  If ppl are still blind. U guys are cracked',
    'end': None,
    'label': '0',
    'probability': 0.9618216,
    'start': None}],
  'task': 'text_classification'},
 {'predictions': [{'context': 'Proven facts are labelled as Conspiracy theories',
    'end': None,
    'label': '0',
    'probability': 0.9510601,
    'start': None},
   {'context': 'You have got to be kidding,

Because the result of the inference test is nested, i create 2 for-loops<br>
(Mir ist aufgefallen, dass immer 4 context Elemente in den predictions Elementen enthalten sind, darum range(3) in der zweiten for-Schleife)

In [ ]:
possible_toxic_comments = []

for x in range(len(result)):
  for y in range(3):
    if result[x]['predictions'][y]['label'] == '1':
      print(result[x]['predictions'][y]['context'])
      possible_toxic_comments.append(result[x]['predictions'][y]['context'])

possible_toxic_comments

What a sad pathetic loser! He has no dignity left and can only be a bitter thorn in America&#39;s side!
U LOST DUDE   Stfu
Pathological liar leading the weak-minded.
Lady speaking, you are going to be puking your guts out when God removes that bitter veil from your eyes.  Surely you would not want your son to become a tyrannical, senile, incompetent leader such as those in the current admin...where are you?  Oh yeah, under a rock with your head in the sand and the other end about to get smacked with the truth.  What a disgraceful news yak session.
He did win you morons.
you stupids need to find a way to get your orange lump sectioned. its getting old
That man has no zero dignity , self respect , pride or integrity ... pathetic &amp; petty sore loser ......<br>Zoe , Canada.
You people that back these thieves of the 2020 election know better a man who stayed in his basement cant recite one sentence without a screw up yall are the most stupid arrogant bunch that has ever drawn a breath th

107

In [ ]:
print(len(result))
print(len(possible_toxic_comments))

107